# Modeling and Simulation in Python

Project 1

Leatherback Sea Turtles Nesting Mother Population

Kyle McCracken and David Tarazi

In [2]:
Question to answer: What is and what will be the leatherback sea turtle population of nesting females in the US virgin islands?

Object `islands` not found.


In [ ]:
Question to answer: What is and what will be the leatherback sea turtle population of nesting females in the US virgin islands

Model:
Parameters/Data
    US Virgin Island Data:
        Nesting numbers, females recorded on the beach
    Data we need to test for:
        population proxy, birth/survival rate for females, natural death rate
Functions Needed with parameters needed from system/state
    births(nestpop, pop proxy)
    step/newnestpop(last years prediction/pop proxy, births-deaths)
        trying to align this prediciton with the females recorded to predict nesting female population
        also uses data to decypher returning females rate after 1yr, 2yrs, 3yrs... until stop
final plot of number of nesting females

In [3]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

from pandas import read_html